# Scoring Script Test
The purpose of this test is a proof of concept implementation of NannyML performance estimation into the bridging scoring script. This could then be used to analyse data drift.

In [1]:
import pickle
import numpy as np
import re
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt;
import warnings; warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pom_NEW import *
import os
import sys
import gcsfs
sys.path.append(os.path.abspath("/home/jupyter/POM-feature-drift"))
from google.cloud import storage
from google.cloud import bigquery
import project_config as pc
import common_variables as cv

In [2]:
table_id = 'offer_bridging_eoo_base'
bucket_location = 'EU'
bucket_id       = 'gs://'+pc.bucket+'/pom_scoring'
file_name       = 'eoo_base_2022-12-05'
file_format     = 'CSV'
gcs_file_path   = os.path.join(bucket_id,file_name+'_*.csv')
prefix_name = 'pom_scoring'

In [3]:
#Connection to BQ
client = bigquery.Client(project=pc.project_id) #;
query = """ SELECT column_name, data_type
FROM """+pc.target_dataset+""".INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'offer_bridging_eoo_base'
AND data_type = 'DATE'
"""

date_cols = client.query(query).to_dataframe().iloc[:, 0].tolist()

In [8]:
#Load columns used in new models - speeds up process as we don't need to keep unused columns
cols = {'Account_Number', 'Earliest_Offer_Week', 'base_dt', 'eoo_base_obs_dt', 'Cohort', 'Rack_Rate', 'Ttl_Offer_Discount', 'Customer_Type', 'Country'}
for customer_type in ['DTV']:
    for country in ['UK']:
        for target_type in ['arpu', 'churn', 'ta']:
            for model_type in ['NT', 'L', 'M', 'H']:
                pickle_name = f'pickle_files/{customer_type}_{country}_{target_type}_{model_type}.pkl'
                if os.path.isfile(pickle_name):
                    with open(pickle_name, 'rb') as pickle_file:
                        model = pickle.load(pickle_file)
                    if (type(model) is XGBRegressor) or (type(model) is XGBClassifier):
                        model_columns = model.get_booster().feature_names
                    elif (type(model) is LGBMRegressor) or (type(model) is LGBMClassifier):
                        model_columns = model.feature_name_
                    elif (type(model) is CombinedModel):
                        model_columns = model.feature_name_
                    else:
                        print(f'MODEL TYPE NOT MATCHED {pickle_name}')
                        model_columns = []
                    if model_columns is None:
                        print(f'NONE COLUMNS {pickle_name}')
                        model_columns = []
                    cols = cols.union(model_columns)
rename_andrew = fix_columns()
cols = cols.union({x for x, y in rename_andrew.items() if y in cols})
cols = cols.union({x.rsplit('_', 1)[0] for x in cols})

In [5]:
#Load scoring data from GCS to pandas dataframe
df_main = read_pomdata_to_score(project = pc.project_id, bucket_name = pc.bucket, prefix_name = prefix_name)

blobs are ['pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000000.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000001.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000002.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000003.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000004.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000005.csv']
Total # of date variables are 167
Inital dataframe contains 456831 rows and 987 columns
Shape of the dataset is 456831 rows and 987 columns


## Clean EOO data

In [9]:
dpp = DataPreProcess(df=df_main.rename(columns={'EOO_Base_Obs_Dt' : 'eoo_base_obs_dt'}), cols=cols, date_cols=date_cols)

eoo_base_obs_dt available in the date df


In [12]:
del df_main

In [13]:
_ = dpp.process_dates()

In [14]:
_, _, _, _ = dpp.fill_missing()

In [15]:
_ = dpp.scale_numeric(excl_cols=['Account_Number', 'Earliest_Offer_Week', 'Movies_Active', 'Sports_Active', 'SGE_Active', 'HD_Active', 'MS_Active'])

In [16]:
_ = dpp.one_hot_encode(nunique=100)

[]


In [17]:
data_consolidated = dpp.concat_data(column_fix={'HD_Product_Holding_nan' : 'HD_Product_Holding_None'})
data_consolidated_scaled = dpp.concat_data(scale_numeric=True, column_fix=rename_andrew)

Data contains the total 456831 rows and 266 columns
Data contains the total 456831 rows and 266 columns


In [18]:
data_consolidated_uk_dtv = data_consolidated.loc[(data_consolidated['Country_UK'] == 1) & (data_consolidated['Customer_Type'] == 'DTV')]
data_consolidated_uk_dtv_scaled = data_consolidated_scaled.loc[(data_consolidated_scaled['Country_UK'] == 1) & (data_consolidated_scaled['Customer_Type'] == 'DTV')]

In [19]:
stepup_models = ['NT', 'L', 'M', 'H']

## Score TA Models

In [20]:
data_dict_uk_dtv_stepup = score_data(df=data_consolidated_uk_dtv_scaled, customer_type='DTV', country='UK', target='target_ta', model_types=stepup_models)

************************Model file DTV_UK_ta_NT exists************************
************************Scored data for model DTV_UK_ta_NT************************
************************Model file DTV_UK_ta_L exists************************
************************Scored data for model DTV_UK_ta_L************************
************************Model file DTV_UK_ta_M exists************************
************************Scored data for model DTV_UK_ta_M************************
************************Model file DTV_UK_ta_H exists************************
************************Scored data for model DTV_UK_ta_H************************


In [21]:
scored_df_ta_uk_dtv_new = pd.concat(list(data_dict_uk_dtv_stepup.values()), axis=1)
scored_df_ta = pd.concat([scored_df_ta_uk_dtv_new])

In [22]:
del scored_df_ta_uk_dtv_new

In [23]:
data_consolidated_uk_dtv_scaled.head()

,Earliest_Offer_Week,Lima_ICD_Flag,Account_Number,Rack_Rate,Offer_Discount_SoD,Ttl_Offer_Discount,ARPU,Prev_Offer_Amount_LR,Curr_Offer_Amount_LR,Curr_Offer_Amount_HD,Offers_Applied_Lst_36M_HD,Curr_Offer_Amount_HD_Pack,Curr_Offer_Amount_SKY_BOX_SETS,Curr_Offer_Amount_SKY_KIDS,Contract_Offers_Applied_Lst_12M_DTV,Curr_Contract_Offer_Amount_BB,Curr_Promo_Offer_Amount_DTV,DTV_Activations_In_Last_3Yr,BB_Subscriber_Activations_In_Last_3Yr,BB_Churns_In_Last_3Yr,MS_Active,HD_Active,SGE_Active,TAs_in_last_24m,Age,Prev_Offer_Amount_DTV,Curr_Offer_Amount_DTV,Offers_Applied_Lst_24M_DTV,Prev_Offer_Amount_BB,Prev_Offer_Amount_Movies,Curr_Offer_Amount_Movies,Sports_Active,Sports_Product_Count,Movies_Active,OD_DLs_Completed_In_Last_7d,OD_DLs_Completed_In_Last_30d,OD_DLs_Completed_In_Last_60d,Sky_Consumer_Market_Share,Virgin_Consumer_Market_Share,Throughput_Speed,max_speed_uplift,cancels_3mth,cancels_6mth,cancels_9mth,cancels_12mth,mobile_3mth,mobile_6mth,mobile_9mth,bill_6mth,signin_6mth,vip_6mth,vip_12mth,help_6mth,total_viewing_duration_overlap_3m,total_viewing_duration_overlap_2m,total_viewing_duration_overlap_1m,linear_viewing_total_overlap_1m,linear_viewing_total_overlap_3m,Last_Offer_Applied_Dt_HD_monthdiff,Last_Contract_Offer_Applied_Dt_DTV_monthdiff,Last_Promo_Offer_Applied_Dt_BB_monthdiff,DTV_Last_Activation_Dt_monthdiff,BB_Last_Activation_Dt_monthdiff,BB_Last_Churn_Dt_monthdiff,DTV_Last_PC_Effective_To_Dt_monthdiff,DTV_Last_Active_Block_Dt_monthdiff,DTV_Last_AB_Effective_To_Dt_monthdiff,last_TA_dt_monthdiff,Last_Value_Call_Dt_monthdiff,Last_Platform_Retention_Call_Dt_monthdiff,Last_Service_Call_Dt_monthdiff,Last_All_Call_Dt_monthdiff,Curr_Offer_Start_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_BB_monthdiff,Last_Completed_OD_DL_Dt_monthdiff,Last_Credit_Dt_monthdiff,Customer_Type,BB_Product_Holding_12GB,BB_Product_Holding_BroadbandEssential20ADSL,BB_Product_Holding_BroadbandSuperfast40FTTC,BB_Product_Holding_BroadbandSuperfast80FTTC,BB_Product_Holding_Connect,BB_Product_Holding_ConnectLiteROILegacy,BB_Product_Holding_ConnectUnlimitedROILegacy,BB_Product_Holding_ConnectUnlimitedROI,BB_Product_Holding_EssentialROI,BB_Product_Holding_EssentialPlus,BB_Product_Holding_Everyday,BB_Product_Holding_FibreROILegacy,BB_Product_Holding_FibreLite,BB_Product_Holding_FibreMax,BB_Product_Holding_FibreUltra350Mb,BB_Product_Holding_FibreUnlimitedROILegacy,BB_Product_Holding_FibreUnlimitedPlus,BB_Product_Holding_FibreUnlimitedPro,BB_Product_Holding_FibreUnlimitedProFTTC,BB_Product_Holding_FibreUnlimitedProFTTP,BB_Product_Holding_GigafastFTTP,BB_Product_Holding_GigafastROI,BB_Product_Holding_SKYFibreMaxSOGEA,BB_Product_Holding_SKYFibreUnlimitedSOGEA,BB_Product_Holding_SkyBroadbandLite,BB_Product_Holding_SkyBroadbandLiteROILegacy,BB_Product_Holding_SkyFibre,BB_Product_Holding_SkySuperfastSFTC,BB_Product_Holding_SkyUltrafastMaxROI,BB_Product_Holding_SkyUltrafastPlusROI,BB_Product_Holding_SuperfastFTTP,BB_Product_Holding_SuperfastROI,BB_Product_Holding_Superfast35FTTC,BB_Product_Holding_Superfast35FTTP,BB_Product_Holding_Superfast35SOGEA,BB_Product_Holding_UltrafastROI,BB_Product_Holding_Ultrafast1FTTP,BB_Product_Holding_Ultrafast160SOGfast,BB_Product_Holding_UltrafastPlusFTTP,BB_Product_Holding_UltrafastPlusROI,BB_Product_Holding_Unlimited,BB_Product_Holding_UnlimitedLegacy,BB_Product_Holding_UnlimitedROILegacy,BB_Product_Holding_UnlimitedFibre,BB_Product_Holding_UnlimitedPro,BB_Product_Holding_nan,Talk_Product_Holding_Anytime,Talk_Product_Holding_AnytimeROI,Talk_Product_Holding_Freetime,Talk_Product_Holding_FreetimeROI,Talk_Product_Holding_Other,Talk_Product_Holding_SkyPayAsYouTalk,Talk_Product_Holding_SkyTalkAnytimeExtra,Talk_Product_Holding_SkyTalkAnytimeExtra247,Talk_Product_Holding_SkyTalkAnytimeExtraAnytimeUK,Talk_Product_Holding_SkyTalkAnytimeExtraAnytime,Talk_Product_Holding_SkyTalkAnytimeExtraVoIP,Talk_Product_Holding_SkyTalkEveningsandWeekendsExtra,Talk_Product_Holding_SkyTalkInternationalExtra,Talk_Product_Holding_W

## NannyML Performance Estimation Implementation

### Setup TA Data

In [24]:
ta_data = data_consolidated_uk_dtv_scaled.join(scored_df_ta, on = 'Account_Number')

In [45]:
ta_data.rename(columns = {'pred_ta_NT': 'pred_proba_ta_NT', 'pred_ta_L': 'pred_proba_ta_L', 'pred_ta_M': 'pred_proba_ta_M', 'pred_ta_H': 'pred_proba_ta_H'}, inplace = True)
for treatment in stepup_models:   
    threshold = ta_data[f'pred_proba_ta_{treatment}'].mean()
    pred = (ta_data[f'pred_proba_ta_{treatment}'] >= threshold).astype(int).to_list()
    ta_data[f'pred_ta_{treatment}'] = pred

In [32]:
dates = ta_data.Earliest_Offer_Week.to_list()
timestamps = []
for i, date in enumerate(dates):
    timestamps.append(pd.to_datetime(str(dates[i]) + '-0', format = '%Y%W-%w'))

In [33]:
ta_data['timestamp'] = timestamps

In [47]:
ta_data.sort_values(by = 'timestamp', inplace = True)

In [35]:
ta_model_dict = {}
for treatment in stepup_models:
    with open(f'pickle_files/DTV_UK_ta_{treatment}.pkl', 'rb') as pkl_file:
        ta_model_dict[treatment] = pickle.load(pkl_file)

### Fit and run performance estimation and pickle outputs

In [52]:
import nannyml as nml

In [72]:
for ref, treatment in zip(['nt', 'low', 'medium', 'high'], stepup_models):   
    ref_data = pd.read_csv(f'data/ta_{ref}_reference_full.csv', low_memory = False)
    ref_data['timestamp'] = pd.to_datetime(ref_data['timestamp'])
    ref_data.sort_values(by = 'timestamp', inplace = True)
    features = ta_model_dict[treatment].feature_name_

    estimator = nml.CBPE(y_pred_proba = 'pred_proba_ta',
                         y_pred = 'pred_ta',
                         y_true = 'target_ta',
                         metrics = ['roc_auc'],
                         timestamp_column_name = 'timestamp',
                         # chunk_period = 'W',
                         chunk_number = 10,
                         problem_type = 'classification_binary'
                        )

    estimator = estimator.fit(ref_data[features + ['pred_proba_ta', 'pred_ta', 'target_ta', 'timestamp']])
    estimated_performance = estimator.estimate(ta_data.rename(columns = {f'pred_proba_ta_{treatment}': 'pred_proba_ta', f'pred_ta_{treatment}': 'pred_ta'})[features + ['pred_proba_ta', 'pred_ta', 'timestamp']])
    
    with open(f'data/NannyML_results/ta_{ref}_perf_est_results.pkl', 'wb') as pkl_file:
        pickle.dump(estimated_performance, pkl_file, protocol = pickle.HIGHEST_PROTOCOL)

In [76]:
# A DataFrame containing mean ROC_AUC for each model. In this test we will display this DataFrame, but in production we could export this as a
# BigQuery table for reference while also triggering another phase (such as data drift calcualtions) based on alerts (requires threshold tests).
# NB: Will use the performance estimation on the reference data as a baseline, this is because the algorithm is good at detecting changes
#     but it is hard to know how accurate it is compared to the realised value. This can be investigated further on new data.

perf_est_results = pd.DataFrame(columns = ['EOO_Base', 'Model', 'Mean_Reference_ROC_AUC', 'Mean_Analysis_ROC_AUC', 'Percentage_Decrease', 'Alert'])
perf_est_results = perf_est_results.assign(Model = stepup_models)
perf_est_results = perf_est_results.assign(EOO_Base = file_name.split('_')[2])

threshold = 0.68

for i, model in enumerate(['nt', 'low', 'medium', 'high']):
    with open(f'data/NannyML_results/ta_{model}_perf_est_results.pkl', 'rb') as pkl_file:
        results = pickle.load(pkl_file)
        results_df = results.to_df(multilevel = False)
        
    perf_est_results.iloc[i, 2] = results_df[results_df.chunk_period == 'reference'].roc_auc_value.mean()
    perf_est_results.iloc[i, 3] = results_df[results_df.chunk_period == 'analysis'].roc_auc_value.mean()
    perf_est_results.iloc[i, 4] = ((perf_est_results.iloc[i, 2] - perf_est_results.iloc[i, 3]) / perf_est_results.iloc[i, 2]) * 100
    perf_est_results.iloc[i, 5] = perf_est_results.iloc[i, 3] < threshold

In [77]:
perf_est_results

,EOO_Base,Model,Mean_Reference_ROC_AUC,Mean_Analysis_ROC_AUC,Percentage_Decrease,Alert
0,2022-12-21,NT,0.754522,0.685289,9.175726,False
1,2022-12-21,L,0.742374,0.676349,8.893779,True
2,2022-12-21,M,0.744044,0.683979,8.07271,False
3,2022-12-21,H,0.741983,0.658106,11.304394,True


## Score ARPU Models

In [78]:
data_dict_uk_dtv_stepup = score_data(df=data_consolidated_uk_dtv_scaled, customer_type='DTV', country='UK', target='target_arpu', model_types=stepup_models)

************************Model file DTV_UK_arpu_NT exists************************
************************Scored data for model DTV_UK_arpu_NT************************
************************Model file DTV_UK_arpu_L exists************************
************************Scored data for model DTV_UK_arpu_L************************
************************Model file DTV_UK_arpu_M exists************************
************************Scored data for model DTV_UK_arpu_M************************
************************Model file DTV_UK_arpu_H exists************************
************************Scored data for model DTV_UK_arpu_H************************


In [79]:
scored_df_arpu_uk_dtv_new = pd.concat(list(data_dict_uk_dtv_stepup.values()), axis=1)
scored_df_arpu = pd.concat([scored_df_arpu_uk_dtv_new])

In [80]:
del scored_df_arpu_uk_dtv_new

## NannyML Performance Estimation Implementation

### Setup TA Data

In [81]:
arpu_data = data_consolidated_uk_dtv_scaled.join(scored_df_arpu, on = 'Account_Number')

In [82]:
arpu_data.head()

,Earliest_Offer_Week,Lima_ICD_Flag,Account_Number,Rack_Rate,Offer_Discount_SoD,Ttl_Offer_Discount,ARPU,Prev_Offer_Amount_LR,Curr_Offer_Amount_LR,Curr_Offer_Amount_HD,Offers_Applied_Lst_36M_HD,Curr_Offer_Amount_HD_Pack,Curr_Offer_Amount_SKY_BOX_SETS,Curr_Offer_Amount_SKY_KIDS,Contract_Offers_Applied_Lst_12M_DTV,Curr_Contract_Offer_Amount_BB,Curr_Promo_Offer_Amount_DTV,DTV_Activations_In_Last_3Yr,BB_Subscriber_Activations_In_Last_3Yr,BB_Churns_In_Last_3Yr,MS_Active,HD_Active,SGE_Active,TAs_in_last_24m,Age,Prev_Offer_Amount_DTV,Curr_Offer_Amount_DTV,Offers_Applied_Lst_24M_DTV,Prev_Offer_Amount_BB,Prev_Offer_Amount_Movies,Curr_Offer_Amount_Movies,Sports_Active,Sports_Product_Count,Movies_Active,OD_DLs_Completed_In_Last_7d,OD_DLs_Completed_In_Last_30d,OD_DLs_Completed_In_Last_60d,Sky_Consumer_Market_Share,Virgin_Consumer_Market_Share,Throughput_Speed,max_speed_uplift,cancels_3mth,cancels_6mth,cancels_9mth,cancels_12mth,mobile_3mth,mobile_6mth,mobile_9mth,bill_6mth,signin_6mth,vip_6mth,vip_12mth,help_6mth,total_viewing_duration_overlap_3m,total_viewing_duration_overlap_2m,total_viewing_duration_overlap_1m,linear_viewing_total_overlap_1m,linear_viewing_total_overlap_3m,Last_Offer_Applied_Dt_HD_monthdiff,Last_Contract_Offer_Applied_Dt_DTV_monthdiff,Last_Promo_Offer_Applied_Dt_BB_monthdiff,DTV_Last_Activation_Dt_monthdiff,BB_Last_Activation_Dt_monthdiff,BB_Last_Churn_Dt_monthdiff,DTV_Last_PC_Effective_To_Dt_monthdiff,DTV_Last_Active_Block_Dt_monthdiff,DTV_Last_AB_Effective_To_Dt_monthdiff,last_TA_dt_monthdiff,Last_Value_Call_Dt_monthdiff,Last_Platform_Retention_Call_Dt_monthdiff,Last_Service_Call_Dt_monthdiff,Last_All_Call_Dt_monthdiff,Curr_Offer_Start_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_BB_monthdiff,Last_Completed_OD_DL_Dt_monthdiff,Last_Credit_Dt_monthdiff,Customer_Type,BB_Product_Holding_12GB,BB_Product_Holding_BroadbandEssential20ADSL,BB_Product_Holding_BroadbandSuperfast40FTTC,BB_Product_Holding_BroadbandSuperfast80FTTC,BB_Product_Holding_Connect,BB_Product_Holding_ConnectLiteROILegacy,BB_Product_Holding_ConnectUnlimitedROILegacy,BB_Product_Holding_ConnectUnlimitedROI,BB_Product_Holding_EssentialROI,BB_Product_Holding_EssentialPlus,BB_Product_Holding_Everyday,BB_Product_Holding_FibreROILegacy,BB_Product_Holding_FibreLite,BB_Product_Holding_FibreMax,BB_Product_Holding_FibreUltra350Mb,BB_Product_Holding_FibreUnlimitedROILegacy,BB_Product_Holding_FibreUnlimitedPlus,BB_Product_Holding_FibreUnlimitedPro,BB_Product_Holding_FibreUnlimitedProFTTC,BB_Product_Holding_FibreUnlimitedProFTTP,BB_Product_Holding_GigafastFTTP,BB_Product_Holding_GigafastROI,BB_Product_Holding_SKYFibreMaxSOGEA,BB_Product_Holding_SKYFibreUnlimitedSOGEA,BB_Product_Holding_SkyBroadbandLite,BB_Product_Holding_SkyBroadbandLiteROILegacy,BB_Product_Holding_SkyFibre,BB_Product_Holding_SkySuperfastSFTC,BB_Product_Holding_SkyUltrafastMaxROI,BB_Product_Holding_SkyUltrafastPlusROI,BB_Product_Holding_SuperfastFTTP,BB_Product_Holding_SuperfastROI,BB_Product_Holding_Superfast35FTTC,BB_Product_Holding_Superfast35FTTP,BB_Product_Holding_Superfast35SOGEA,BB_Product_Holding_UltrafastROI,BB_Product_Holding_Ultrafast1FTTP,BB_Product_Holding_Ultrafast160SOGfast,BB_Product_Holding_UltrafastPlusFTTP,BB_Product_Holding_UltrafastPlusROI,BB_Product_Holding_Unlimited,BB_Product_Holding_UnlimitedLegacy,BB_Product_Holding_UnlimitedROILegacy,BB_Product_Holding_UnlimitedFibre,BB_Product_Holding_UnlimitedPro,BB_Product_Holding_nan,Talk_Product_Holding_Anytime,Talk_Product_Holding_AnytimeROI,Talk_Product_Holding_Freetime,Talk_Product_Holding_FreetimeROI,Talk_Product_Holding_Other,Talk_Product_Holding_SkyPayAsYouTalk,Talk_Product_Holding_SkyTalkAnytimeExtra,Talk_Product_Holding_SkyTalkAnytimeExtra247,Talk_Product_Holding_SkyTalkAnytimeExtraAnytimeUK,Talk_Product_Holding_SkyTalkAnytimeExtraAnytime,Talk_Product_Holding_SkyTalkAnytimeExtraVoIP,Talk_Product_Holding_SkyTalkEveningsandWeekendsExtra,Talk_Product_Holding_SkyTalkInternationalExtra,Talk_Product_Holding_W

In [83]:
dates = arpu_data.Earliest_Offer_Week.to_list()
timestamps = []
for i, date in enumerate(dates):
    timestamps.append(pd.to_datetime(str(dates[i]) + '-0', format = '%Y%W-%w'))

In [84]:
arpu_data['timestamp'] = timestamps

In [85]:
arpu_data.sort_values(by = 'timestamp', inplace = True)

In [86]:
arpu_model_dict = {}
for treatment in stepup_models:
    with open(f'pickle_files/DTV_UK_arpu_{treatment}.pkl', 'rb') as pkl_file:
        arpu_model_dict[treatment] = pickle.load(pkl_file)

### Fit and run performance estimation and pickle outputs

In [52]:
import nannyml as nml

In [87]:
ref_data = pd.read_csv(f'data/arpu_low_reference_full.csv', low_memory = False)
ref_data.head()

,ttl_offer_discount,hd_active,last_value_call_dt_monthdiff,arpu,bill_6mth,sge_product_holding_SGEPaid,curr_offr_sub_type_sports_SPORTS,hd_product_holding_HDPremiumRose,curr_offer_amount_hd,dtv_last_activation_dt_monthdiff,prev_offer_amount_bb,od_dls_completed_in_last_7d,age,curr_offer_amount_dtv,hd_product_holding_HDBasicSkyHD,prev_offer_amount_dtv,prev_offer_amount_lr,cancels_6mth,bb_product_holding_FibreMax,total_viewing_duration_overlap_3m,prev_offer_amount_movies,last_completed_od_dl_dt_monthdiff,last_ta_dt_monthdiff,last_offer_applied_dt_dtv_monthdiff,offers_applied_lst_24m_dtv,sports_active,bb_churns_in_last_3yr,sports_product_count,dtv_last_pc_effective_to_dt_monthdiff,last_all_call_dt_monthdiff,dtv_last_active_block_dt_monthdiff,last_credit_dt_monthdiff,curr_offer_amount_lr,help_6mth,movies_active,target_arpu,pred_arpu,timestamp
0,1.055032,-0.615035,-0.716845,1.739859,-0.335977,0.0,1.0,0.0,0.399885,0.976477,-0.116742,-0.424997,0.084351,0.114128,0.0,0.361325,0.550239,-0.15821,0.0,-0.608489,0.332683,-0.265168,-1.025435,-0.159449,0.520016,1.134924,-0.367462,1.197728,0.490994,-0.147388,0.466259,-0.698655,0.234856,-0.148002,1.150738,53.96,44.372485,2020-11-15
1,-1.639302,-0.615035,-0.721276,-1.547508,-0.335977,0.0,0.0,0.0,0.399885,-0.370888,0.941967,-0.465137,0.117611,0.972713,0.0,0.702834,0.550239,-0.15821,0.0,-0.608489,0.553395,-0.264359,-1.024433,-0.156079,0.520016,-0.881116,-0.367462,-0.851140,-2.037509,-0.145365,-2.139901,-0.697800,0.234856,-0.343634,-0.869007,8.00,3.327915,2020-11-15
2,-0.959292,-0.615035,-0.716212,-0.291994,-0.335977,0.0,0.0,0.0,0.399885,0.179560,-0.116742,-0.465137,0.450214,0.114128,0.0,-0.592322,-0.647325,-0.15821,0.0,-0.608489,0.553395,3.776617,-1.024834,-0.152035,-0.486714,-0.881116,-0.367462,-0.851140,0.490994,-0.153457,0.466259,-0.697800,0.234856,-0.343634,-0.869007,22.00,3.394033,2020-11-15
3,-0.975877,-0.615035,1.385105,-1.733235,-0.335977,0.0,0.0,0.0,0.399885,-0.568063,0.941967,-0.144012,-0.431183,0.114128,0.0,-0.347467,0.550239,-0.15821,0.0,-0.141614,0.553395,-0.265168,-1.030248,-0.152035,0.520016,-0.881116,-0.367462,-0.851140,0.490994,-0.148062,0.466259,1.430729,0.234856,-0.343634,-0.869007,11.00,11.610799,2020-11-15
4,-1.224662,-0.615035,1.385105,-1.621799,-0.335977,0.0,0.0,0.0,0.399885,-0.707729,0.941967,-0.465137,0.300543,0.114128,0.0,1.005681,0.550239,-0.15821,0.0,0.145906,0.553395,-0.264763,0.971897,-0.152035,-0.486714,-0.881116,-0.367462,-0.851140,0.490994,6.585348,0.466259,1.430729,0.234856,-0.343634,-0.869007,11.00,10.499653,2020-11-15


In [91]:
for ref, treatment in zip(['nt', 'low', 'medium', 'high'], stepup_models):   
    features = arpu_model_dict[treatment].feature_name_
    
    with open(f'data/NannyML_results/arpu_{ref}_perf_estimator.pkl', 'rb') as pkl_file:
        estimator = pickle.load(pkl_file)
    
    estimated_performance = estimator.estimate(arpu_data.rename(columns = {f'pred_arpu_{treatment}': 'pred_arpu'})[features + ['pred_arpu', 'timestamp']])
    
    with open(f'data/NannyML_results/arpu_{ref}_perf_est_results.pkl', 'wb') as pkl_file:
        pickle.dump(estimated_performance, pkl_file, protocol = pickle.HIGHEST_PROTOCOL)

In [97]:
# A DataFrame containing mean RMSE and MAE for each model. In this test we will display this DataFrame, but in production we could export this as a
# BigQuery table for reference while also triggering another phase (such as data drift calcualtions) based on alerts (requires threshold tests).
# NB: Will use the performance estimation on the reference data as a baseline, this is because the algorithm is good at detecting changes
#     but it is hard to know how accurate it is compared to the realised value. This can be investigated further on new data.

perf_est_results = pd.DataFrame(columns = ['EOO_Base', 'Model', 'Mean_Reference_RMSE', 'Mean_Analysis_RMSE', 'Percentage_Increase_RMSE',
                                           'Mean_Reference_MAE', 'Mean_Analysis_MAE', 'Percentage_Increase_MAE', 'Alert'])
perf_est_results = perf_est_results.assign(Model = stepup_models)
perf_est_results = perf_est_results.assign(EOO_Base = file_name.split('_')[2])

threshold = 70

for i, model in enumerate(['nt', 'low', 'medium', 'high']):
    with open(f'data/NannyML_results/arpu_{model}_perf_est_results.pkl', 'rb') as pkl_file:
        results = pickle.load(pkl_file)
        results_df = results.to_df(multilevel = False)
        
    perf_est_results.iloc[i, 2] = results_df[results_df.chunk_period == 'reference'].rmse_value.mean()
    perf_est_results.iloc[i, 3] = results_df[results_df.chunk_period == 'analysis'].rmse_value.mean()
    perf_est_results.iloc[i, 4] = ((perf_est_results.iloc[i, 3] - perf_est_results.iloc[i, 2]) / perf_est_results.iloc[i, 2]) * 100
    
    perf_est_results.iloc[i, 5] = results_df[results_df.chunk_period == 'reference'].mae_value.mean()
    perf_est_results.iloc[i, 6] = results_df[results_df.chunk_period == 'analysis'].mae_value.mean()
    perf_est_results.iloc[i, 7] = ((perf_est_results.iloc[i, 6] - perf_est_results.iloc[i, 5]) / perf_est_results.iloc[i, 5]) * 100
    
    perf_est_results.iloc[i, 8] = perf_est_results.iloc[i, 3] > threshold

In [98]:
perf_est_results

,EOO_Base,Model,Mean_Reference_RMSE,Mean_Analysis_RMSE,Percentage_Increase_RMSE,Mean_Reference_MAE,Mean_Analysis_MAE,Percentage_Increase_MAE,Alert
0,2022-12-21,NT,56.639052,72.941511,28.783071,41.005987,53.157469,29.633435,True
1,2022-12-21,L,49.300304,67.032658,35.968042,34.817726,52.764772,51.545716,False
2,2022-12-21,M,47.965032,72.559257,51.275323,32.713846,50.026223,52.920644,True
3,2022-12-21,H,47.062986,66.237056,40.741295,32.044561,45.690779,42.585132,False
